In [ ]:
import numpy as np

class Softmax2CrossEntropy():
    def __init__(self):
        self.y    = None
        self.t    = None
        self.flag = False

    def forward(self, x, t):
        self.y = np.empty(x.shape)
        self.t = t
        
        for i in range(self.y.shape[0]):
            c = np.amax(x[i])
            self.y[i] = np.exp(x[i] - c)/(np.sum(np.exp(x[i] - c)))

        self.flag = True
        
        return -(np.sum(self.t*np.log(self.y)))/self.t.shape[0]

    def test_forward(self, x):
        y = np.empty(x.shape)

        for i in range(y.shape[0]):
            c = np.amax(x[i])
            y[i] = np.exp(x[i] - c)/(np.sum(np.exp(x[i] - c)))
        
        return y

    def test_loss(self, x, t):
        y = self.test_forward(x)
        return -(np.sum(t*np.log(y)))/t.shape[0]

    def backward(self):
        if self.flag:
            self.flag = False
            return (self.y - self.t)/self.t.shape[0]
        
        else:
            raise Exception("you have to run forward method before")
            
class Swish():
    def __init__(self):
        self.x    = None
        self.flag = False

    def forward(self, x):
        self.x    = x
        self.flag = True
        return self.x/(1 + np.exp(-self.x))

    def test_forward(self, x):
        return x/(1 + np.exp(-x))

    def backward(self, delta):
        if self.flag:
            self.flag = False
            return delta*((1 + np.exp(-self.x)*(1 + self.x))/((1 + np.exp(-self.x))**2))
        
        else:
            raise Exception("you have to run forward method before")

class ReLU():
    def __init__(self):
        self.x    = None
        self.flag = False

    def forward(self, x):
        self.x    = x
        self.flag = True
        return np.maximum(0, self.x)

    def test_forward(self, x):
        return np.maximum(0, x)

    def backward(self, delta):
        if self.flag:
            self.flag = False
            return delta*((self.x > 0).astype(np.int))
        
        else:
            raise Exception("you have to run forward method before")

class Sigmoid():
    def __init__(self):
        self.x    = None
        self.flag = False

    def forward(self, x):
        self.x    = x
        self.flag = True
        return 1/(1 + np.exp(-self.x))

    def test_forward(self, x):
        return 1/(1 + np.exp(-x))

    def backward(self, delta):
        if self.flag:
            self.flag = False
            return delta*((np.exp(-self.x))/(1 + np.exp(-self.x))**2)
        
        else:
            raise Exception("you have to run forward method before")

class Affine():
    def __init__(self, inputSize, outputSize):
        self.data  = {"IN":inputSize, "OUT":outputSize}
        self.W     = np.random.randn(self.data["IN"], self.data["OUT"])*0.1
        self.b     = np.random.randn(self.data["OUT"])*0.1
        self.dW    = np.zeros_like(self.W)
        self.db    = np.zeros_like(self.b)
        self.delta = None
        self.x     = None
        self.flag  = False 


    def forward(self, x):
        self.x    = x
        self.flag = True

        return np.dot(self.x, self.W) + self.b

    def test_forward(self, x):
        return np.dot(x, self.W) + self.b

    def backward(self, delta):
        if self.flag:
            self.flag = False
            self.delta = delta
            self.dW = np.dot(self.x.T, self.delta)
            self.db = np.sum(self.delta, axis = 0)

            return np.dot(self.delta, self.W.T)
        
        else:
            raise Exception("you have to run forward method before")

class BatchNorm():
    def __init__(self, inputSize):
        self.W      = np.ones(inputSize)
        self.b      = np.zeros(inputSize)
        self.mu     = None
        self.sigma  = None
        self.m      = None
        self.dW     = None
        self.db     = None
        self.x      = None
        self.x_hat  = None
        self.delta  = None
        self.xc     = None
        self.std    = None
        
        self.flag   = False

    def forward(self, x):
        self.x     = x
        self.m     = self.x.shape[0]
        self.flag  = True

        self.mu    = np.sum(self.x, axis = 0)/self.m
        self.xc    = self.x - self.mu
        self.sigma = np.sum((self.xc)**2, axis = 0)/self.m
        self.std   = np.sqrt(self.sigma + 1e-7)
        self.x_hat = self.xc/self.std

        return self.W*self.x_hat + self.b

    def test_forward(self, u):
        x     = u
        m     = x.shape[0]

        mu    = np.sum(x, axis = 0)/m
        sigma = np.sum((x - mu)**2, axis = 0)/m
        x_hat = ((x - mu)/np.sqrt(sigma + 1e-7))

        return self.W*x + self.b

    def backward(self, delta):
        if self.flag:
            self.flag    = False

            self.db      = np.sum(delta, axis = 0)
            self.dW      = np.sum(delta*self.x_hat, axis = 0)

            dxn  = self.W*delta
            dxc  = dxn/self.std
            dstd = -np.sum((dxn*self.xc)/(self.std*self.std), axis = 0)
            dvar = 0.5*dstd/self.std
            dxc += (2.0/self.m)*self.xc*dvar
            dmu  = np.sum(dxc, axis = 0)
            dx   = dxc - dmu/self.m

            return dx 

            """
            f0           = self.sigma + 1e-7
            f1           = self.x - self.mu
            f2           = np.sum(-f1, axis = 0)/self.m
            f3           = 2*(f1 + f2)/self.m
            f4           = 1/(2*np.sqrt(f0))
            f5           = f3*f4

            row_size1    = f5.shape[0]
            column_size1 = f5.shape[1]
            f6           = f5.T.flatten()

            f7           = np.zeros(column_size1*row_size1*column_size1)
            indx1       = (np.arange(f5.size)*column_size1 + np.floor(np.arange(f5.size)/row_size1)).astype(int)
            f7[indx1]    = f6
            f8           = f7.reshape(column_size1, row_size1, column_size1)
            f9           = np.dot(f8, f1.T)

            f10          = np.identity(self.m) - 1/self.m
            f11          = np.resize(f10, (f0.shape[0], f10.shape[0], f10.shape[1]))
            f12          = np.sqrt(f0)
            f13          = f12[:, np.newaxis, np.newaxis]
            f14          = f13*f11

            f15          = f14 - f9
            f16          = (1/f0)[:, np.newaxis, np.newaxis]
            f17          = f16*f15
            f18          = np.dot(f17, delta)

            row_size2    = f18.shape[1]
            column_size2 = f18.shape[2]
            indx2        = (np.arange(row_size2*column_size2)*column_size2 + np.floor(np.arange(row_size2*column_size2)/row_size2)).astype(int)
            f19          = f18.flatten()
            f20          = f19[indx2]
            f21          = f20.reshape(column_size2, row_size2).T + 0
            
            return f21
            """

        else:
            raise Exception("you have to run forward method before")